# Introduction

In this notebook, I performed exploratory data analysis of the Covid 19 Case Surveillance Public Use Dataset, and then used the data to create models to predict patient outcomes given the types of data included in the dataset.

## Findings

### Missing Data
I began by looking at the unique values inputted for each variable.  I found there were very high rates of data entries of "Missing" and "Unknown".  I knew rows with missing values must be removed for some pieces of analysis, but I found this curious, so I investigated which patients had the most missing values. 

I found that less data was missing for older patients compared to younger patients. I suspect the reason for this is related to severity of disease cases, as well as perhaps the uncertainty of young patients having relevant pre-existing medical conditions. 

I also found differences in the amount of missing data when looking at patient race/ethnicity.  Here, I supposed the missing data may be related to regional hospital practices, or hospital stress (i.e., being under-staffed). 

### Predictive Models

I continued on to create predictive models.  I used the caret package to make training and testing datasets from the patients who did not have missing/unknown entries.  I compared three different models in predicting patient hospitalization.  The models were base R binomial GLM, caret Bayes GLM, and caret rpart decision tree.  I compared these models by their accuracy, their ROC curves and AUCs, and their run-times.  While both GLM models had similar ROC curves and accuracies, the base R binomial GLM is significantly faster.  Therefore, I continued with this modeling technique to predict patient death.  

The hospitalization predictions had a 83% accuracy, and death predictions had ~95% accuracy.  This held fairly constant for the training and testing sets.  I have concluded it is a reasonable model for the data.

## Additional Considerations

Upon learning that a friend of mine had a recent exposure to Covid-19, I decided to run the models on his case.  It seems worthwhile to mention here, as I told my friend, that this data set is limited.  This is only data for patients for who there is data.  There is no data on the large number of asymptomatic and very mild cases among people who never thought to get a Covid-19 test or seek medical attention.  Therefore, the probability of an outcome such as hospitalization or death is very likely lower than that which any of these models can predict.

In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
library(tictoc)
tic()
data <- read.csv('../input/covid19-case-surveillance-public-use-dataset/COVID-19_Case_Surveillance_Public_Use_Data.csv')
#head(data)
dat <- data[, 4:11]
#head(dat)
toc()

In [ ]:
# rename columns for ease
colnames(dat) <- c("current_status", "sex", "age", "race_ethnicity", "hosp", "icu", "death", "medcond")

# determine unique vals in each column to identify those without information, or of insufficient frequency (e.g., sex == "Other")
for (col in 1:8){
    print(colnames(dat)[col])
    print(as.data.frame(table(dat[,col])))
}

In [ ]:
# Make Dataframe without Missing/Unknown/Low-count values
df <- dat %>% filter(sex %in% c("Female", "Male"),
                     age != "Unknown",
                     race_ethnicity != "Unknown" & race_ethnicity != "Missing",
                     hosp %in% c("Yes", "No"),
                     icu %in% c("Yes", "No"),
                     death %in% c("Yes", "No"),
                     medcond %in% c("Yes", "No"))
for (col in 1:8){
    print(colnames(df)[col])
    print(as.data.frame(table(df[,col])))
}

In [ ]:

dim(df)
#dim(dat)



This filtration reduced the dataframe from 8e6 to 4e5 rows.  The remaineder is still sufficient to perform analysis.

However, this is a dramatic reduction in data.  Which demographic groups have lost the most entries?

In [ ]:
# investigate age bias
Afiltering <- cbind(as.data.frame(table(dat$age)),as.data.frame(table(df$age))[,2] )
colnames(Afiltering) <- c("age", "Freq1", "Freq2" )
Afiltering$percent_lost <- 100 * (1 - Afiltering[,3] / Afiltering[,2])
Afiltering

In [ ]:
Akeeplose <- cbind(Afiltering$Age, Afiltering[,3], #keep
                    (Afiltering[,2] - Afiltering[,3]))[-10,] #lost, and removing 'Unknown' row
Akeeplose
prop.test(as.matrix(Akeeplose), correct = F)

In [ ]:
# investigate race_ethnic bias
REfiltering <- cbind(as.data.frame(table(dat$race_ethnicity)),as.data.frame(table(df$race_ethnicity))[,2] )
colnames(REfiltering) <- c("race_ethnicity", "Freq1", "Freq2" )
REfiltering$percent_lost <- 100 * (1 - REfiltering[,3] / REfiltering[,2])
REfiltering


In [ ]:
Afilter<- Afiltering[1:9, c(1,4)]
REfilter <- REfiltering[c(-5,-8), c(1,4)]
Afilter
REfilter

In [ ]:
RE_outcomes <- dat %>% group_by(race_ethnicity) %>% summarise(p_hosp = 100*mean(hosp == "Yes"),
                                                              p_icu = 100*mean(icu== "Yes"),
                                                              p_death = 100*mean(death == "Yes"))

Age_outcomes <- dat %>% group_by(age) %>% summarise(p_hosp = 100*mean(hosp == "Yes"), 
                                                    p_icu = 100*mean(icu== "Yes"),
                                                    p_death = 100*mean(death == "Yes"))
RE_outcomes <- as.data.frame(RE_outcomes)[c(-5,-8,-10),]
Age_outcomes <- as.data.frame(Age_outcomes)[1:9,]
RE_outcomes
Age_outcomes

In [ ]:
RE_outcomes <- merge(RE_outcomes, REfilter, by = "race_ethnicity")
Age_outcomes <- merge(Age_outcomes, Afilter, by = "age")
RE_outcomes
Age_outcomes

In [ ]:
# Plot age & outcomes
a <- Age_outcomes
plot(a$percent_lost, a$p_hosp, type = "b", col = "green", lwd = 2,
     xlab = "Percent with Missing/Unknown Data", 
     ylab = "Percent of Total With Outcome", main = "Patient Outcomes by Age and Percent Missing Data", 
    ylim=c(-1, 30))
lines(a$percent_lost, a$p_icu, type = "b", lwd = 2,col = "red")
lines(a$percent_lost, a$p_death, type = "b", lwd = 2,col = "black")
legend(93.6,28, legend=c("Hospital", "ICU", "Death"),
       col=c("green", "red", "black"), lty=1, bty="n", cex = 1.3)
abline(v=a$percent_lost, col=c("grey"), lty=c(1))
coords <- c(5, 16, 12, 20, 7,9, 5, 15, 11)
for (i in 1:9){
    text(a$percent_lost[i], coords[i], toString(a$age[i]), pos = c(4,3,2,4,2,3,3,3,3), srt=90)
}

# Plot Race/Ethnicity & Outcomes
r <- RE_outcomes[order(RE_outcomes$percent_lost),]
plot(r$percent_lost, r$p_hosp, type = "b", col = "green", lwd = 2,
     xlab = "Percent with Missing/Unknown Data", 
     ylab = "Percent of Total With Outcome", 
    ylim=c(-1, 18),
    main = "Patient Outcomes by Race/Ethnicity and Percent Missing Data",)
lines(r$percent_lost, r$p_icu, type = "b", lwd = 2,col = "red")
lines(r$percent_lost, r$p_death, type = "b", lwd = 2,col = "black")
legend(95,19, legend=c("Hospital", "ICU", "Death"),
       col=c("green", "red", "black"), lty=1, bty="n")
abline(v=r$percent_lost, col=c("grey"), lty=c(1))
#coords <- c(5, 15, 12, 20, 8,9, 5, 15, 11)
for (i in 1:9){
    text(r$percent_lost[i], r$p_hosp[i], toString(r$race_ethnicity[i]), pos = c(4,4,4,4,1,4,4), srt=90, cex = .7)
}

Less data was acquired in younger patients.  Perhaps this was due to less information about, say, pre-existing medical conditions, and fewer younger patients required the more advanced care.

We can see here there is a negative relationship between age groups with missing entries in the data and hospital/ICU/death rates.

However, when considering race/ethnic groups, there is not such a relationship.  Some race/ethnic groups had significantly more missing data than others.  Disease outcomes, most notably hospitalization, also varied greatly across race/ethnic groups. 

Some patterns of note:
* People categorized as Asian or Black have higher rates of poor disease outcomes.
* People categorized as Native Hawaiian/Pacific Islander have lower rates of missing data.  This may be due to lower rates of covid in Hawaii, resulting in a better capacity for hospital staff to record more details of patients.
* People categorized as American Indian/ Alaskan Native had the highest rate of missing data, at 97%, more than 2% more than the next group (race/ethnicity recorded as Multiple/Other). This may also be connected to regional hospital staff capacity to record detailed data.



The proportion test provides a p-value of 2.2e-16, which suggests that the differences in race/ethnic groups' rates of missing data are not due to chance.  Of course, this result is not surprising, as the identical investigation into differences in data completion for age groups found a differences within 1-2% to also not be due to chance.  However, it is noteworthy that the data discrepancies between age groups are much smaller than those for ethnic groups.  Additionally, the age groups most at risk for disease complications (older decades) had more thorough records, while the opposite is true regarding risk level and race/ethnicity (see models below).

Possible explanations for missing data:
* communication/language barriers
* hospital recording methods
* hospital recording dilligence (keeping in mind how many hospitals have been short of staff, etc.)

It is unfortunate to have such discrepancies in the data.  We may consider minimizing these.  There is more 'Missing' or 'Unknown' data from the ICU variable and the MedCond variable than any other.  

Regarding the ICU variable, I suspect that different hospitals had different methods for whether or not this data was recorded/included.  Next I will investigate whether the ICU variable provides more information than Hosp or Death.  In other words, does the ICU data have a strong additional influence on predictions?

Regarding Medcond, there is no alternative to this information, but later we can investigate if it is a strong predictor of patient outcomes.



In [ ]:
# Compare Hosp, ICU, & Death for rows in which there is no missing data.
HID <- df[,5:7]
#unique(HID) # check all entries are 'Yes' or 'No'
HID <- ifelse(HID == 'Yes', 1, 0)

cor(HID[,1], HID[,2])
cor(HID[,2], HID[,3])
cor(HID[,1], HID[,3])

We find a modest correlation between hosp & ICU and between death & ICU.  There is an even lower correlation between hosp and death (which is intuitive).  However, this leaves a moderate predictive power for hosp or death to take the place of the missing ICU entries.

Given this, I will continue with the analysis only for the non-missing/unknown data.  I will do a similar investigation with the removal of ICU and Medcond as predictors to include a larger amount of data.

In [ ]:
## https://datascienceplus.com/perform-logistic-regression-in-r/

library(caret)
#df <- ifelse(df[5:7] == 'Yes', 1, 0)



In [ ]:
# factorize variables to remove "Missing" and "Unknown" levels
for (i in 1:8){
    df[,i] <- factor(df[,i])
}


In [ ]:
head(df)

In [ ]:
# predict hosp
set.seed(100)
intrain<-createDataPartition(y=df$hosp,p=0.7,list=FALSE)
training<-df[intrain,]
testing<-df[-intrain,]


In [ ]:
# caret bayesglm model

# this takes a long time to run
# age 20 - 69 -higher pval
# 70s, 80s strongest predictor (by coefficient)
# all race_eth negative coef besides Native, which is also the group with the 
# least detailed data collection.  Perhaps outcomes are more closely related to hospital conditions 
# compared to race_ethn.  Similarly, being white reduces the most
tic()
bayes_hosp_model <- train(hosp ~ age + sex + race_ethnicity + medcond - 1, 
                 method = "bayesglm",data=training)
toc() # ~200 seconds
# this does nothing, but why?  preobj <- preProcess(training[,-5],method = c("pca"), thresh = .9)
#preobj$numComp

summary(bayes_hosp_model)
# model accuracy
confusionMatrix(predict(bayes_hosp_model, training[,]), reference=training$hosp) # Accuracy 84%

In [ ]:
# decision tree 
# note: cannot run random forest on my computer
tic()
caret_rpart_hosp_model <- train(hosp ~ age + sex + race_ethnicity + medcond - 1, 
                 method = "rpart",data=training)
toc() # ~100 sec
#summary(caret_rpart_hosp_model)

confusionMatrix(predict(caret_rpart_hosp_model, training[,]), reference=training$hosp) #also ~84%


In [ ]:
# GLM predict hosp
tic()
glm_hosp_model <- glm(hosp ~ factor(age) + factor(sex) + factor(race_ethnicity) + medcond - 1, 
                  family="binomial",data=training)
toc() # ~2 sec
summary(glm_hosp_model)

#test_glm_hosp <- predict(glm_hosp_model, testing[,], type = "response")
train_pred <- predict(glm_hosp_model, training[,-5], type = "response")
#summary(train_pred)

binary_train_pred <- ifelse(train_pred > 0.5,1,0)
#summary(train_pred)


binary_train_hosp <- ifelse(training$hosp == "Yes",1,0)
levels(binary_train_hosp) <- c(0,1) 
levels(binary_train_pred) <- c(0,1) 

binary_train_hosp <- factor(binary_train_hosp)
binary_train_pred <- factor(binary_train_pred)

confusionMatrix(binary_train_pred, reference=binary_train_hosp) #also ~84%

In [ ]:
# ROC evaluation
library(ROCR)


In [ ]:
# GLM ROC
pr <- prediction(train_pred, binary_train_hosp)
prf <- performance(pr, measure = "tpr", x.measure = "fpr")
plot(prf, main = "GLM ROC, AUC = .828, Accuracy = 84%")

auc <- performance(pr, measure = "auc")
auc <- auc@y.values[[1]]
auc # 0.828

In [ ]:
# rpart Decision Tree ROC

dt_pred <- predict(caret_rpart_hosp_model, training[,], type = "prob")
#head(dt_pred)

dt_pr <- prediction(dt_pred[,2],training$hosp)
dt_prf <- performance(dt_pr, measure = "tpr", x.measure = "fpr")
plot(dt_prf, main = "Decision Tree ROC, AUC = .745, Accuracy = 84%")

dt_auc <- performance(dt_pr, measure = "auc")
dt_auc <- dt_auc@y.values[[1]]
dt_auc # 0.745


In [ ]:
# caret Bayes GLM ROC

bayes_pred <- predict(bayes_hosp_model, training[,], type = "prob")
#head(bayes_pred)

bayes_pr <- prediction(bayes_pred[,2],training$hosp)
bayes_prf <- performance(bayes_pr, measure = "tpr", x.measure = "fpr")
plot(bayes_prf, main = "Bayes GLM ROC, AUC = .828, Accuracy = 84%")

b_auc <- performance(bayes_pr, measure = "auc")
b_auc <- b_auc@y.values[[1]]
b_auc # 0.828


In [ ]:
plot( bayes_prf, col = "blue", lwd = 2, alpha = .8, main = "ROC Comparisons: All Models Had Accuracy ~84%")
plot(prf, add = TRUE, col= "orange", lty= "dashed", lwd = 2, alpha = .2)
plot(dt_prf, add = TRUE, col= "green", lwd = 2)
legend(0.2, 0.4, legend=c("Bayes GLM: AUC = .83", "GLM: AUC = .83", "Dec Tree: AUC = .75"),
       col=c("blue", "orange", "green"), lty=c(1,1,1), lwd = 2, bty = "n", cex = 1.5)

The two GLM models perform equally well, and are better than the Decision Tree for predicting a hospital outcome from demographic and medical condition data.  The Bayes GLM model under the caret package had a processing time of approximately 200 seconds.  The other GLM model had a processing time of two seconds, so this is a more efficient way to generate the linear model.

Continuing with this GLM model, we can evaluate how data on medical conditions and ICU stays influences the model in predicting patient death.

In [ ]:
head(training)

In [ ]:
# GLM predict death

# predict with hosp, icu, & medcond
tic()
glm_death_model <- glm(death ~ factor(age) + factor(sex) + factor(race_ethnicity) + hosp + icu+ medcond - 1, 
                  family="binomial",data=training)
toc() # ~4 sec
#summary(glm_death_model)

train_pred <- predict(glm_death_model, training[,-7], type = "response")
#summary(train_pred)

binary_train_pred <- ifelse(train_pred > 0.5,1,0)
#summary(train_pred)


binary_train_death <- ifelse(training$death == "Yes",1,0)
levels(binary_train_death) <- c(0,1) 
levels(binary_train_pred) <- c(0,1) 

binary_train_death <- factor(binary_train_death)
binary_train_pred <- factor(binary_train_pred)

confusionMatrix(binary_train_pred, reference=binary_train_death) #95% accuracy, Sensitivity : 0.980, Specificity : 0.5195 

In [ ]:
# GLM predict death

# predict with hosp, without icu, & medcond
glm_death_model_2 <- glm(death ~ factor(age) + factor(sex) + factor(race_ethnicity) + hosp  - 1, 
                  family="binomial",data=training)


train_pred <- predict(glm_death_model_2, training[,-7], type = "response")
#summary(train_pred)

binary_train_pred <- ifelse(train_pred > 0.5,1,0)
#summary(train_pred)


binary_train_death <- ifelse(training$death == "Yes",1,0)
levels(binary_train_death) <- c(0,1) 
levels(binary_train_pred) <- c(0,1) 

binary_train_death <- factor(binary_train_death)
binary_train_pred <- factor(binary_train_pred)

confusionMatrix(binary_train_pred, reference=binary_train_death) #94% accuracy, Sensitivity : 0.9805, Specificity : 0.3576 

In [ ]:
par(mfrow = c(2, 2))
plot(glm_death_model)
par(mfrow = c(2, 2))
plot(glm_hosp_model)

## Applying Models to Test Sets

In [ ]:
# Bayes GLM test
bayes_test <- predict(bayes_hosp_model, testing[,], type = "prob")
confusionMatrix(predict(bayes_hosp_model, testing[,]), reference=testing$hosp) # Accuracy 83%


In [ ]:
htest <- predict(glm_hosp_model, testing[,-5], type = "response")

binary_htest <- ifelse(htest > 0.5,1,0)

binary_testing_h <- ifelse(testing$hosp == "Yes",1,0)
levels(binary_htest) <- c(0,1) 
levels(binary_testing_h) <- c(0,1) 

binary_htest <- factor(binary_htest)
binary_testing_h <- factor(binary_testing_h)

confusionMatrix(binary_htest, reference=binary_testing_h) #also ~84%

In [ ]:
d1test <- predict(glm_death_model, testing[,-7], type = "response")
binary_d1test <- ifelse(d1test > 0.5,1,0)

binary_testing_d <- ifelse(testing$death == "Yes",1,0)
levels(binary_d1test) <- c(0,1) 
levels(binary_testing_d) <- c(0,1) 

binary_d1test <- factor(binary_d1test)
binary_testing_d <- factor(binary_testing_d)

confusionMatrix(binary_d1test, reference=binary_testing_d) # accuracy 95%

d2test <- predict(glm_death_model_2, testing[,-7], type = "response")
binary_d2test <- ifelse(d2test > 0.5,1,0)
levels(binary_d2test) <- c(0,1) 
binary_d2test <- factor(binary_d2test)
confusionMatrix(binary_d2test, reference=binary_testing_d) #also ~94%




## Conclusion

The GLM models for predicting hospitalization and death all performed well on the test sets.  Specificity values ranged from around 30% to 50%.  Sensitivity values were all at least 95%.  

The GLM model is highly predictive.  ICU and medcond data is beneficial in predicting death as a patient outcome.  While it is the case that these factors were often not included in patients' data, it is slightly beneficial for predicting future patient outcomes to keep record of this data.

The GLM model from the caret package was similarly predictive to the base R GLM model, however, the latter is a signficantly faster, and therefore a better choice with this data.



In [ ]:
# Prediction out of curiousity for my friend with a recent covid exposure:

Michael_data <- testing[14750,] # matches Michael info (sex, age, race)
Michael_data_medcond <- testing[14760,] # matches Michael + medcond to give more conservative prediction

print("hospitalization probabilities without and with a medical condition")
predict(bayes_hosp_model, Michael_data, type = "prob")

predict(bayes_hosp_model, Michael_data_medcond, type = "prob")

print("death probabilities without and with a medical condition")

predict(glm_death_model_2, Michael_data, type = "response")
predict(glm_death_model_2, Michael_data_medcond, type = "response") #interestingly, identical to prediction without medcond, ~0.2%
